In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os

def imshow(img, size=(5, 5)):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.rcParams['figure.figsize'] = size
    plt.imshow(img)
    plt.yticks([])
    plt.xticks([])
    plt.show()

IMAGE_PATH = "../data/static/"

imgs = [cv.imread(IMAGE_PATH + img) for img in os.listdir(IMAGE_PATH) if img.split('.')[-1] in ['jpg', 'jpeg', 'png']]

## Experiment with edge detection

In [ ]:
def process_image(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    sharrx = cv.Scharr(gray, cv.CV_8U, 1, 0)
    sharry = cv.Scharr(gray, cv.CV_8U, 0, 1)
    mag_sharr, angle_sharr = cv.cartToPolar(sharrx.astype('float32'), sharry.astype('float32'), angleInDegrees=True)
    mag_sharr_norm = cv.normalize(mag_sharr, None, 0, 255, cv.NORM_MINMAX).astype(np.uint8)

    sobelx = cv.Sobel(gray, cv.CV_8U, 1, 0)
    sobely = cv.Sobel(gray, cv.CV_8U, 0, 1)
    mag_sob, angle_sob = cv.cartToPolar(sobelx.astype('float32'), sobely.astype('float32'), angleInDegrees=True)
    mag_sob_norm = cv.normalize(mag_sob, None, 0, 255, cv.NORM_MINMAX).astype(np.uint8)
    
    imshow(img)
    imshow(np.concatenate([
        np.concatenate([sharrx, sharry, mag_sharr_norm], axis=1),
        np.concatenate([sobelx, sobely, mag_sob_norm], axis=1)],
        axis=0
    ), size=(30,11))

for img in imgs:
    process_image(img)

Clearly edge detection seems to work better with sobel than with sharr

## Let's see speed of this code

In [ ]:
%%timeit -n 100
gray = cv.cvtColor(imgs[0], cv.COLOR_BGR2GRAY)
sobelx = cv.Sobel(gray, cv.CV_8U, 1, 0)
sobely = cv.Sobel(gray, cv.CV_8U, 0, 1)
mag_sob, angle_sob = cv.cartToPolar(sobelx.astype('float32'), sobely.astype('float32'), angleInDegrees=True)

# Canny

In [ ]:
def canny(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    edges = cv.Canny(gray, 100, 300, apertureSize=3)
    imshow(np.concatenate([gray, edges], axis=1), size=(30,11))

for img in imgs:
    canny(img)

## Custom filter
Try to focus on thin lines. \
Let's see what happens after applying sobel on both positive and negative directions.

In [ ]:
gray = cv.cvtColor(imgs[2], cv.COLOR_BGR2GRAY)

sobel_kernel = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])
cut = gray[400:500, 500:600]
down = cv.filter2D(cut, cv.CV_8U, sobel_kernel.T) # compues gradient in positive y direction
up = cv.filter2D(cut, cv.CV_8U, np.flip(sobel_kernel.T)) # compues gradient in negative y direction
imshow(np.concatenate([cut, down, up], axis=1), size=(30, 11))

Thin objects (like lines) will have high values near to each other in both directions. \
Let's use thresholding, dilation and logical AND to get only those pixels.

In [ ]:
thup = cv.threshold(up, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
thdown = cv.threshold(down, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
dup = cv.dilate(thup, np.ones((3,3), np.uint8))
ddown = cv.dilate(thdown, np.ones((3,3), np.uint8))
AND = cv.bitwise_and(dup, ddown)
imshow(np.concatenate([thup, thdown], axis=1), size=(10, 11))
imshow(np.concatenate([dup, ddown, AND], axis=1), size=(30, 11))

As you can see most stones are gone, but some are still there.
Let's how this works if we apply it on a whole image and both horizontally and vertically.

In [ ]:
def line_filter(gray):
    def line_filter_aux(img, kernel):
        down = cv.filter2D(gray, cv.CV_8U, kernel)
        up = cv.filter2D(gray, cv.CV_8U, np.flip(kernel))
        thup = cv.threshold(up, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
        thdown = cv.threshold(down, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
        dup = cv.dilate(thup, np.ones((3,3), np.uint8))
        ddown = cv.dilate(thdown, np.ones((3,3), np.uint8))
        AND = cv.bitwise_and(dup, ddown)
        return AND

    sobel_kernel = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ])

    a = line_filter_aux(gray, sobel_kernel.T)
    b = line_filter_aux(gray, sobel_kernel)
    AND = cv.bitwise_and(a, b) 
    OR = cv.bitwise_or(a, b)
    return AND, OR

for img in imgs:
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    AND, OR = line_filter(gray)
    imshow(np.concatenate([gray, AND, OR]), size=(30, 15))

My idea was to use logical OR to get all intersections of lines, however it doesn't necessarily work good on all images. \
We will stick with using **AND** for now and try to use it to segment the board

In [ ]:
%%timeit -n 100
line_filter(gray)